In [ ]:
from numba import jit
from numba.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)


@jit(nopython=True)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


@jit(nopython=True)
def hit_rate(train_triples, test_triples, a, b, c,
             how_many=10, iter_show=False, freq=3000):
    
    total = len(test_triples)
    hit = 0
    iteration = 0
    for entity in test_triples:
        iteration += 1
        p = entity[0]
        q = entity[1]
        r = entity[2]

        candidate_values = np.sum(a[p, :] * b[q, :] * c, axis=1)
        candidate_values = sigmoid(candidate_values)
        
        top = np.argsort(candidate_values)[::-1]
        ltop = [i for i in top]
        

        for tr in train_triples:    
            if (p == tr[0]) and (q == tr[1]) and (r != tr[2]):
                if tr[2] in ltop:
                    ltop.remove(tr[2])
        
        ltop = ltop[:how_many]
        
        for x in ltop:
            if x == r:
                hit += 1
                
        if iter_show:
            if iteration % freq == 0:
                print(hit / iteration, hit, iteration)
            
    return hit / total, hit, total


@jit(nopython=True)
def hit_rate_raw(test_triples, a, b, c,
             how_many=10, iter_show=False, freq=3000):
    
    total = len(test_triples)
    hit = 0
    iteration = 0
    for entity in test_triples:
        iteration += 1
        p = entity[0]
        q = entity[1]
        r = entity[2]

        candidate_values = np.sum(a[p, :] * b[q, :] * c, axis=1)
        candidate_values = sigmoid(candidate_values)
        
        top = np.argsort(candidate_values)[::-1][:how_many]
        
        for x in top:
            if x == r:
                hit += 1
                
        if iter_show:
            if iteration % freq == 0:
                print(hit / iteration, hit, iteration)
            
    return hit / total, hit, total

hit_rate(
    train_triples[:5],
    test_triples[:5],
    a, b, c, 10,
    iter_show=False,
    freq=3000,
)